In [11]:
import pandas as pd
import requests_html
from bs4 import BeautifulSoup
import datetime
import time

In [12]:
file_name = "Peertec 메타마스크 Polygon 네트워크 입출금"
df = pd.read_excel(file_name+".xls")
df.head(5)

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(MATIC),Value_OUT(MATIC),CurrentValue @ $0.6656/MATIC,TxnFee(MATIC),TxnFee(USD),Historical $Price/MATIC,Status,ErrCode,Method
0,0xbc4ff4b35db6e69fb0c53cc08e4c85505b527dae4bd7...,27862943,1651556220,2022-05-03 05:37:00,0x6e685a45db4d97ba160fa067cb81b40dfed47245,0x81251d5047f57bb6158d39b72ae1bf92faf26261,NaN,12000.0,0.0,7987.200,0.010500,0.006989,1.08,NaN,NaN,Transfer
1,0xc817ea045d1471ec6eb12e34626f48899c7be1dd762c...,27862987,1651556312,2022-05-03 05:38:32,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,NaN,0.0,5990.0,3986.944,0.008807,0.005862,1.08,NaN,NaN,Swap Exact ETH For Tokens
2,0xb6bf57fdfe61eb96bdf1fbc473261c8f90953a972784...,27863057,1651556456,2022-05-03 05:40:56,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,NaN,0.0,5990.0,3986.944,0.016562,0.011024,1.08,NaN,NaN,Swap Exact ETH For Tokens
3,0xe3c75046a1d8dedc53cbab994e8ec883dc01bc567465...,27863147,1651556640,2022-05-03 05:44:00,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,NaN,0.0,0.0,0.000,0.001858,0.001237,1.08,NaN,NaN,Approve
4,0xdfaf19d8aca694abc8f4b52ffc7c976816da7ad2de79...,27863165,1651556676,2022-05-03 05:44:36,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,NaN,0.0,0.0,0.000,0.001634,0.001088,1.08,NaN,NaN,Approve


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Txhash                        48 non-null     object        
 1   Blockno                       48 non-null     int64         
 2   UnixTimestamp                 48 non-null     int64         
 3   DateTime                      48 non-null     datetime64[ns]
 4   From                          48 non-null     object        
 5   To                            48 non-null     object        
 6   ContractAddress               0 non-null      float64       
 7   Value_IN(MATIC)               48 non-null     float64       
 8   Value_OUT(MATIC)              48 non-null     float64       
 9   CurrentValue @ $0.6656/MATIC  48 non-null     float64       
 10  TxnFee(MATIC)                 48 non-null     float64       
 11  TxnFee(USD)                   48 n

In [14]:
# Null 처리
df = df[df['Status'].isnull()==True]

# KST Time도 더해줘야지
df['KST Time (GMT+9)'] = df.apply(lambda r: r['DateTime'] + datetime.timedelta(hours=9), axis = 1)

In [15]:
# 한줄이면 가능
df = df.loc[:,['KST Time (GMT+9)','Txhash','Method','From','To','Value_IN(MATIC)','Value_OUT(MATIC)']]
df.head(20)

,KST Time (GMT+9),Txhash,Method,From,To,Value_IN(MATIC),Value_OUT(MATIC)
0,2022-05-03 14:37:00,0xbc4ff4b35db6e69fb0c53cc08e4c85505b527dae4bd7...,Transfer,0x6e685a45db4d97ba160fa067cb81b40dfed47245,0x81251d5047f57bb6158d39b72ae1bf92faf26261,12000.000,0.0
1,2022-05-03 14:38:32,0xc817ea045d1471ec6eb12e34626f48899c7be1dd762c...,Swap Exact ETH For Tokens,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.000,5990.0
2,2022-05-03 14:40:56,0xb6bf57fdfe61eb96bdf1fbc473261c8f90953a972784...,Swap Exact ETH For Tokens,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.000,5990.0
3,2022-05-03 14:44:00,0xe3c75046a1d8dedc53cbab994e8ec883dc01bc567465...,Approve,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0.000,0.0
4,2022-05-03 14:44:36,0xdfaf19d8aca694abc8f4b52ffc7c976816da7ad2de79...,Approve,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,0.000,0.0
5,2022-05-03 14:45:56,0xbdd4610c6beb78371375bc9daeac37fc77473d3941cb...,Add Liquidity,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.000,0.0
6,2022-05-04 11:49:03,0x92dd4636cb74f3d1da328356ce7aebba76315186c49b...,Claim Reward,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x274ebd0a589445d2759e379277984c19dbf83cfd,0.000,0.0
7,2022-05-04 11:49:59,0xcb4de12525d52c1ef5bc62c28609a4a4ea9cf1d8e1a7...,Approve,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x82362ec182db3cf7829014bc61e9be8a2e82868a,0.000,0.0
8,2022-05-04 11:50:27,0xe2b0e83ac91401a65d65e7f1e2c80d7e897a8f4abcb9...,Swap Exact Tokens For Tokens,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.000,0.0
9,2022-05-04 11:51:03,0x9735b846841cc8b00d2deebe0d8ea68678400b76f96c...,Swap Exact ETH For Tokens,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.000,10.0


In [16]:
# Datetime 내림차순 정렬
df.sort_values(by=["KST Time (GMT+9)"], ascending = False, inplace=True)
df.head()

,KST Time (GMT+9),Txhash,Method,From,To,Value_IN(MATIC),Value_OUT(MATIC)
47,2022-05-10 13:16:19,0x9f062f2fbe27b87f8a88446fb2092e15570f3453b6c0...,Transfer,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x8e59558b73a92572834169dc0096df9e42168b57,0.0,571680.0
46,2022-05-10 13:15:01,0xdd550f7d7f6491305275e20b7166405be69e2d4b2fe7...,Swap Exact Tokens For ETH,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0
45,2022-05-10 12:22:10,0xac7b23711b7a5dd7a626c2bdb7c4206bb384fb9496fc...,Swap Exact Tokens For Tokens,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0
44,2022-05-10 12:18:24,0x1779b146c78b67660f7fa5dcebf6d58b114541575545...,Swap Exact Tokens For ETH,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0
43,2022-05-10 12:17:00,0xb06e0329e529bba34f7ae756442e03948f18a9634a3f...,Swap Exact Tokens For ETH,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0


In [17]:
# 4. 타겟 지갑 주소를 from, to와 비교한다
main_wallet_address = "0x8f27fa136ff4da1796368380019029d81d12d06d"
# 돌아가면서 하려면 새로운 열을 만드는게 좋을듯
target_address = 0
# 두번에 돌아가서 
df['TargetAddress'] = df.apply(lambda r: r['To'] if r['From'] == main_wallet_address else r['From'], axis = 1)

# df['From']
# df['To']
df.head(5)


,KST Time (GMT+9),Txhash,Method,From,To,Value_IN(MATIC),Value_OUT(MATIC),TargetAddress
47,2022-05-10 13:16:19,0x9f062f2fbe27b87f8a88446fb2092e15570f3453b6c0...,Transfer,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x8e59558b73a92572834169dc0096df9e42168b57,0.0,571680.0,0x81251d5047f57bb6158d39b72ae1bf92faf26261
46,2022-05-10 13:15:01,0xdd550f7d7f6491305275e20b7166405be69e2d4b2fe7...,Swap Exact Tokens For ETH,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0,0x81251d5047f57bb6158d39b72ae1bf92faf26261
45,2022-05-10 12:22:10,0xac7b23711b7a5dd7a626c2bdb7c4206bb384fb9496fc...,Swap Exact Tokens For Tokens,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0,0x81251d5047f57bb6158d39b72ae1bf92faf26261
44,2022-05-10 12:18:24,0x1779b146c78b67660f7fa5dcebf6d58b114541575545...,Swap Exact Tokens For ETH,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0,0x81251d5047f57bb6158d39b72ae1bf92faf26261
43,2022-05-10 12:17:00,0xb06e0329e529bba34f7ae756442e03948f18a9634a3f...,Swap Exact Tokens For ETH,0x81251d5047f57bb6158d39b72ae1bf92faf26261,0x10f4a785f458bc144e3706575924889954946639,0.0,0.0,0x81251d5047f57bb6158d39b72ae1bf92faf26261


In [18]:
df.to_excel(file_name+'_v1.xlsx', index=False)

In [19]:
# Please enable Javascript
# https://theautomatic.net/2019/01/19/scraping-data-from-javascript-webpage-python/

In [20]:
# from selenium import webdriver
# # 비동기 처리

# options = webdriver.ChromeOptions()
# # options.add_argument('headless')
# # driver = webdriver.Chrome('./chromedriver.exe', options=options)
# driver = webdriver.Chrome('./chromedriver.exe')

In [21]:
# etherscan_url = "https://etherscan.io/address/"
# target_address = "0x8dbe9c44a42050da44f075149c1ff49b18bcaf3f"

# driver.get(etherscan_url + target_address)

In [22]:
# from selenium.webdriver.common.by import By
# address_or_contract_selector = driver.find_element(By.CSS_SELECTOR, "#content > div.container.py-3 > div > div.mb-3.mb-lg-0 > h1")
# address_or_contract_selector.text
# # address_or_contract = address_or_contract_selector.text.split()[0]
# # split으로 둘이 나눠버려!